In [81]:
import os
import json

import numpy as np
import pandas


from keras.models import Sequential
from keras.layers import Dense, Conv2D, BatchNormalization, Activation, InputLayer, LeakyReLU, Reshape, Flatten
from keras.layers.pooling import MaxPooling1D, MaxPooling2D, AveragePooling1D, AveragePooling2D

from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping, ModelCheckpoint

from keras.optimizers import Adam

import librosa

In [3]:
def load_index(fold_id, key='train', n=None):
    
    with open(os.path.join(DESTDIR, 'experiment_{:02d}.json'.format(fold_id)), 'r') as fdesc:
        index = json.load(fdesc)[key]
    
    perm = np.random.permutation(len(index))
    if n is not None:
        perm = perm[:n]
        
    index = [index[_] for _ in perm]
    
    return index

In [33]:
def preprocess(X):
    return librosa.logamplitude(X**2, ref_power=np.max, top_db=80)


def load_data(index, n_harm=1):
    
    X, Y = [], []
    for item in index:
        with np.load(os.path.join(DESTDIR, 'features', '{}.npz'.format(item['filename']))) as data:
            if n_harm is None:
                n_harm = data['C'].shape[0]
            X.append(preprocess(data['C'][:n_harm]))
            Y.append(item['classID'])
    
    X = np.asarray(X).swapaxes(3, 1)
    Y = np.asarray(Y)
    
    return X, Y

In [5]:
DESTDIR = '/home/bmcfee/working/UrbanSound8K/'

In [280]:
FOLD_ID = 1

In [281]:
index = load_index(FOLD_ID)

In [282]:
test_index = load_index(FOLD_ID, key='test')

In [465]:
N_HARM = 4

In [466]:
X, Y = load_data(index, n_harm=N_HARM)

In [467]:
Xt, Yt = load_data(test_index, n_harm=N_HARM)

In [468]:
model = Sequential()

model.add(InputLayer(input_shape=X.shape[1:], name='input'))
model.add(BatchNormalization())

model.add(Conv2D(16, (4 - X.shape[3] + 1) * 5, 5,  bias=False))
model.add(BatchNormalization())
model.add(LeakyReLU(alpha=0.1))

model.add(MaxPooling2D(pool_size=(5, 5)))

model.add(Conv2D(32, 5, 5,  bias=False))
model.add(BatchNormalization())
model.add(LeakyReLU(alpha=0.1))

model.add(MaxPooling2D(pool_size=(5, 5)))

model.add(Conv2D(64, 3, model.output_shape[2], bias=False))
model.add(BatchNormalization())
model.add(LeakyReLU(alpha=0.1))


model.add(AveragePooling2D(pool_size=(model.output_shape[1], 1)))
model.add(Flatten())

model.add(Dense(10, activation='softmax'))



In [469]:
model.compile(Adam(), 'categorical_crossentropy', metrics=['accuracy'])

In [470]:
weight_fn = '/home/bmcfee/working/UrbanSound8K/models_{}.hdf5'.format(N_HARM)

In [471]:
history = model.fit(X, to_categorical(Y, nb_classes=10),
                    callbacks=[EarlyStopping('val_loss', patience=10),
                               ModelCheckpoint(weight_fn, save_best_only=True)],
                    validation_split=0.25, batch_size=32, shuffle=True,
                    nb_epoch=30)

Train on 5894 samples, validate on 1965 samples
Epoch 1/30
5894/5894 [==============================] - 38s - loss: 1.1963 - acc: 0.6105 - val_loss: 1.6615 - val_acc: 0.3954
Epoch 2/30
5894/5894 [==============================] - 31s - loss: 0.7779 - acc: 0.7545 - val_loss: 1.0507 - val_acc: 0.6336
Epoch 3/30
5894/5894 [==============================] - 29s - loss: 0.5829 - acc: 0.8195 - val_loss: 1.0753 - val_acc: 0.6977
Epoch 4/30
5894/5894 [==============================] - 28s - loss: 0.4924 - acc: 0.8531 - val_loss: 1.2194 - val_acc: 0.6494
Epoch 5/30
5894/5894 [==============================] - 27s - loss: 0.4074 - acc: 0.8756 - val_loss: 0.9454 - val_acc: 0.6865
Epoch 6/30
5894/5894 [==============================] - 29s - loss: 0.3309 - acc: 0.9031 - val_loss: 0.5752 - val_acc: 0.8087
Epoch 7/30
5894/5894 [==============================] - 27s - loss: 0.3017 - acc: 0.9074 - val_loss: 0.8832 - val_acc: 0.7272
Epoch 8/30
5894/5894 [==============================] - 27s - loss: 0.

In [472]:
model.load_weights(weight_fn)

In [444]:
print(N_HARM)
model.evaluate(Xt, to_categorical(Yt))

1
873/873 [==============================] - 0s     


[1.2636298434158098, 0.63917525800506136]

In [455]:
print(N_HARM)
model.evaluate(Xt, to_categorical(Yt))

2
873/873 [==============================] - 1s     


[1.9734076197341273, 0.64261168412189984]

In [464]:
print(N_HARM)
model.evaluate(Xt, to_categorical(Yt))

3
873/873 [==============================] - 1s     


[1.8012274783638063, 0.61282932437435733]

In [473]:
print(N_HARM)
model.evaluate(Xt, to_categorical(Yt))

4
873/873 [==============================] - 2s     


[1.5287588606721894, 0.67697594529028471]